<a href="https://colab.research.google.com/github/weagan/Tools/blob/main/image_cleanup_no_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
from PIL import Image
import gradio as gr

print("✓ Packages installed successfully")

# ===== IMAGE CLEANUP FUNCTIONS =====

def denoise_image(image, strength=10):
    """Remove noise using OpenCV"""
    img_array = np.array(image)
    denoised = cv2.fastNlMeansDenoisingColored(img_array, None, strength, strength, 7, 21)
    return Image.fromarray(denoised)

def sharpen_image(image, amount=1.5):
    """Sharpen image using unsharp masking"""
    img_array = np.array(image)
    gaussian = cv2.GaussianBlur(img_array, (0, 0), 2.0)
    sharpened = cv2.addWeighted(img_array, amount, gaussian, -(amount-1), 0)
    return Image.fromarray(np.clip(sharpened, 0, 255).astype(np.uint8))

def adjust_brightness_contrast(image, brightness=0, contrast=0):
    """Adjust brightness and contrast"""
    img_array = np.array(image).astype(np.float32)

    # Apply brightness
    img_array = img_array + brightness

    # Apply contrast
    if contrast != 0:
        factor = (259 * (contrast + 255)) / (255 * (259 - contrast))
        img_array = 128 + factor * (img_array - 128)

    img_array = np.clip(img_array, 0, 255).astype(np.uint8)
    return Image.fromarray(img_array)

def remove_artifacts(image):
    """Remove JPEG artifacts and smooth"""
    img_array = np.array(image)
    # Bilateral filter preserves edges while smoothing
    smoothed = cv2.bilateralFilter(img_array, 9, 75, 75)
    return Image.fromarray(smoothed)

# ===== OBJECT REMOVAL FUNCTION =====

def inpaint_opencv(img_array_src, mask_binary_src):
    """Remove objects using OpenCV inpainting (fast, no AI needed)"""
    # Ensure image is 3-channel for inpainting
    if len(img_array_src.shape) == 2: # Grayscale image
        img_array_src = cv2.cvtColor(img_array_src, cv2.COLOR_GRAY2BGR)
    elif img_array_src.shape[2] == 4: # RGBA image
        img_array_src = cv2.cvtColor(img_array_src, cv2.COLOR_RGBA2RGB)

    # Ensure mask is single channel 8-bit binary (0 or 255)
    # It should already be from process_removal, but as a safeguard
    if len(mask_binary_src.shape) == 3:
        mask_binary_src = cv2.cvtColor(mask_binary_src, cv2.COLOR_RGB2GRAY)
    _, mask_binary_final = cv2.threshold(mask_binary_src, 1, 255, cv2.THRESH_BINARY)

    # Use Telea or NS algorithm for inpainting
    result = cv2.inpaint(img_array_src, mask_binary_final, 3, cv2.INPAINT_TELEA)

    return result # Return numpy array directly

# ===== GRADIO PROCESSING FUNCTIONS =====

def process_cleanup(image, denoise, denoise_strength, sharpen, sharpen_amount,
                   brightness, contrast, remove_jpg_artifacts):
    """Process image cleanup"""
    if image is None:
        return None

    result = Image.fromarray(image)

    if denoise:
        result = denoise_image(result, strength=denoise_strength)

    if remove_jpg_artifacts:
        result = remove_artifacts(result)

    if sharpen:
        result = sharpen_image(result, amount=sharpen_amount)

    if brightness != 0 or contrast != 0:
        result = adjust_brightness_contrast(result, brightness, contrast)

    return np.array(result)

def process_removal(image_dict):
    """Process object removal using OpenCV inpainting"""
    if image_dict is None:
        return None

    # Handle both dict and direct image input from Gradio ImageMask
    if isinstance(image_dict, dict):
        image_np = image_dict.get('background') # This is already a numpy array
        layers = image_dict.get('layers', [])
        mask_layer = layers[0] if layers else None
    else:
        # If image_dict is not a dict, it's likely a direct image input without a mask
        # For this tool, it implies no mask was drawn.
        return image_dict

    if image_np is None:
        return None

    if mask_layer is None:
        return image_np # No mask provided, return original image_np

    # Prepare mask: extract alpha channel and make binary
    mask_array_raw = np.array(mask_layer)
    if mask_array_raw.shape[-1] == 4:  # Has alpha channel (from gradio.ImageMask)
        mask_binary_final = mask_array_raw[:, :, 3] # Extract alpha channel as mask
    else:
        # This case might happen if mask_layer is already 1-channel or something unexpected
        mask_binary_final = np.zeros(image_np.shape[:2], dtype=np.uint8) # Default to empty mask

    if mask_binary_final.max() == 0:
        return image_np  # No mask drawn, return original image_np

    # Call the modified inpaint_opencv, passing numpy arrays directly
    result_np = inpaint_opencv(image_np, mask_binary_final)
    return result_np # Return numpy array

# ===== GRADIO INTERFACE =====

with gr.Blocks(title="Image Cleanup Tool") as demo:
    gr.Markdown("# 🖼️ Image Cleanup & Object Removal Tool")
    gr.Markdown("Fast image processing without heavy AI models!")

    with gr.Tab("✨ Image Cleanup"):
        gr.Markdown("### Clean up and enhance your image")

        with gr.Row():
            with gr.Column():
                input_cleanup = gr.Image(label="Upload Image")

                gr.Markdown("**Noise Removal**")
                denoise_check = gr.Checkbox(label="Remove Noise", value=True)
                denoise_slider = gr.Slider(1, 20, value=10, step=1,
                                          label="Noise Removal Strength")

                gr.Markdown("**Sharpening**")
                sharpen_check = gr.Checkbox(label="Sharpen Image", value=True)
                sharpen_slider = gr.Slider(1.0, 3.0, value=1.5, step=0.1,
                                          label="Sharpening Amount")

                gr.Markdown("**Adjustments**")
                brightness_slider = gr.Slider(-100, 100, value=0, step=1,
                                             label="Brightness")
                contrast_slider = gr.Slider(-100, 100, value=0, step=1,
                                           label="Contrast")
                artifacts_check = gr.Checkbox(label="Remove JPEG Artifacts", value=False)

                cleanup_btn = gr.Button("✨ Clean Up Image", variant="primary")

            with gr.Column():
                output_cleanup = gr.Image(label="Cleaned Image")

        cleanup_btn.click(
            fn=process_cleanup,
            inputs=[input_cleanup, denoise_check, denoise_slider, sharpen_check,
                   sharpen_slider, brightness_slider, contrast_slider, artifacts_check],
            outputs=output_cleanup
        )

    with gr.Tab("🗑️ Remove Objects"):
        gr.Markdown("### Remove unwanted parts of your image")
        gr.Markdown("**Instructions:** Upload an image, then draw over areas you want to remove with the brush tool")
        gr.Markdown("⚠️ Note: Uses fast OpenCV inpainting. For complex removals, results may vary.")

        with gr.Row():
            with gr.Column():
                input_removal = gr.ImageMask(
                    label="Upload & Draw on Areas to Remove",
                    type="numpy"
                )
                removal_btn = gr.Button("🗑️ Remove Marked Areas", variant="primary")

            with gr.Column():
                output_removal = gr.Image(label="Result")

        removal_btn.click(
            fn=process_removal,
            inputs=input_removal,
            outputs=output_removal
        )

✓ Packages installed successfully


In [ ]:

# Launch the interface
print("\n" + "="*50)
print("Starting interface...")
print("="*50 + "\n")

demo.launch(share=True, debug=True)


Starting interface...

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://bec7c66276daa047f5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
